In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from dotenv import load_dotenv


In [ ]:
#import getpass
#os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')


In [2]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')


In [3]:
llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=1000, temperature=0)


In [4]:
# `ChatMessageHistory` is a class from the LangChain library that is used to store and manage the sequence of messages exchanged in a conversation.
# It keeps track of both user and AI messages, allowing the conversational agent to maintain context across multiple turns.
# In this notebook, `ChatMessageHistory` is used to store the chat history for each session, so the AI can refer back to previous messages and provide more coherent and context-aware responses.

store = {}

def get_chat_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [ ]:
chain = prompt | llm

In [8]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [9]:
session_id = "user_123"


response1 = chain_with_history.invoke(
    {"input": "Hello! How are you?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response1.content)

response2 = chain_with_history.invoke(
    {"input": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response2.content)

Parent run 267b3ec5-7077-4d80-bcff-01657e1e8c34 not found for run 85a5dda8-2a7c-4ff0-941c-6a1b83ce42c6. Treating as a root run.
Parent run d67d87c8-182e-4f7b-8df4-fd4312c761ef not found for run eb46f8d6-f728-4008-b0ae-a88cf3447a70. Treating as a root run.


AI: Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
AI: Your previous message was, "Hello! How are you?" How can I assist you further?


In [10]:
print("\nConversation History:")
for message in store[session_id].messages:
    print(f"{message.type}: {message.content}")


Conversation History:
human: Hello! How are you?
ai: Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
human: What was my previous message?
ai: Your previous message was, "Hello! How are you?" How can I assist you further?
